In [12]:
#--imports---
import os
import scanpy as sc
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
import numpy as np


In [ ]:
#---load data and Neuronal Networks---
#set base path to load data: goes back one directory and then into the data
base_path = os.path.join('..', 'data')
#Name of data set
dataset_name = 'large_atac_gene_activity'
# read dataset into an anndata object:  Category - Cells of the brain
inPath = os.path.join(base_path, f"{dataset_name}.h5ad")
adata = sc.read(inPath)

#set base path to load data: goes back one directory and then into the data
base_path = os.path.join('..', 'src', 'models', 'saved_models')
#Name of autoencoder
dataset_name = 'autoencoder_mselossfunction'
# load autoencoder
inPath = os.path.join(base_path, f"{dataset_name}.keras")
autoencoder = tf.keras.models.load_model(inPath)

#Name of discriminator
dataset_name = 'discriminator_pretrained'
# load discriminator
inPath = os.path.join(base_path, f"{dataset_name}.keras")
discriminator = tf.keras.models.load_model(inPath)

In [ ]:
#---Prepare Data---
#ADATA->NUMPY
X_numpy = adata.X.toarray()
#Encode the cell labels as One hot vector to use as additional information
encoder = OneHotEncoder(sparse_output=False)  # `sparse=False` returns a dense array
encoded_labels = encoder.fit_transform(adata.obs[['final_cell_label']])

# Concatenate gen expreesion matrix with oneHotLabels
INPUT = np.concatenate((X_numpy, encoded_labels), axis=1)

#One-hot encoded Batches
encoder = OneHotEncoder(sparse_output=False)  # `sparse=False` returns a dense array
OUTPUT = encoder.fit_transform(adata.obs[['batchname_all']])


In [27]:
results = discriminator(INPUT)
entropy = tf.keras.losses.categorical_crossentropy(OUTPUT, results)

In [31]:
X_numpy.shape

(84813, 3580)

In [49]:
#Wrapper to pass additional things into the loss function
def fool_discriminator_loss(discriminator,encoded_labels, expected_output):
#Define a loss function
    def lossfunction(y_true, y_pred): #y_true Input into autoencoder(ignored here), y_pred: output of autoencoder
        #Prepare Data for Discriminator:
        # Concatenate y_pred with labels using TensorFlow
        INPUT = tf.concat([y_pred, encoded_labels], axis=1)
        discriminator_predictions = discriminator(INPUT)
        # Calculate categorical cross entropy between the discriminator's predictions and the real results 
        CCE = tf.keras.losses.categorical_crossentropy(expected_output, discriminator_predictions)
        #take the inverse of the CCE so that the cross entropy of the discriminator is maximised
        CCEi = 1/CCE
        return CCEi
    return lossfunction

# Compile your autoencoder with the new loss
autoencoder.compile(
    optimizer=autoencoder.optimizer,  # Keep the same optimizer
    loss=fool_discriminator_loss(discriminator, encoded_labels, OUTPUT),
    metrics=autoencoder.metrics
)

# Continue training your generator
autoencoder.fit(
    INPUT, INPUT,
    epochs=10,
    batch_size=30
)

MemoryError: Unable to allocate 1.13 GiB for an array with shape (84813, 3587) and data type float32

In [46]:
autoencoder.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 256)            │       918,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 11)             │         1,419 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,858,531 (10.90 MB)

 Trainable params: 952,843 (3.63 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,905,688 (7.27 MB)